In [1]:
import winsound
import time

def done():
    frequency = 1000  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    
    
    for i in range(5):
        winsound.Beep(frequency, duration)
        time.sleep(0.1)
    



In [2]:
import numpy as np
import os
import random
import spacy
import scispacy
nlp = spacy.load("en_core_sci_sm")
def word2vec(x):
    return nlp(x).vector

<h1>Importing Proteins and Common English Words</h1>

In [3]:


if os.path.isfile('dataset.npy') and os.path.isfile('proteinVectors.npy') and os.path.isfile('commonVectors.npy'):
    dataset = np.load('dataset.npy', allow_pickle=True)
    proteinVectors = np.load('proteinVectors.npy', allow_pickle=True)
    commonVectors = np.load('commonVectors.npy', allow_pickle=True)
else:
    with open ("protein_list.txt", "r", encoding="utf8") as myfile:
        lines=myfile.readlines()
        myfile.close()

    proteins = [l.rstrip() for l in lines]    
    print(len(proteins),"proteins imported")
    
        
    import nltk
    nltk.download('words')
    from nltk.corpus import words
    word_list = words.words()
    # prints 236736
    
    word_list = random.sample(word_list,len(proteins))   
   
    
    words = [w for w in word_list]
    print(len(words),"words imported")
    
    
    for p in proteins:
        if p in words:
            words.remove(p)
            
    proteinVectors = [word2vec(p) for p in proteins]
    
    commonVectors = [word2vec(w) for w in words]

    dataset = [(p,1) for p in proteinVectors]    
    print("done with converting proteins to vectors")
    
    dataset += [(w,0) for w in commonVectors]
    print("done with converting words to vectors")

    np.save('dataset.npy', dataset)
    np.save('proteinVectors.npy', proteinVectors)
    np.save('commonVectors.npy', commonVectors)


In [4]:
import random

#print(dataset[0])

np.random.shuffle(dataset)

#print(dataset[0])

#dataset = random.sample(dataset,len(dataset))





vectors = [l[0] for l in dataset]
labels = [l[1] for l in dataset]

print(len(vectors))
print(len(labels))
print(round(len(labels)/5))
ratio = round(len(labels)/5)

print(vectors[0].shape)

for i,k in enumerate(labels):
    print(i,k)
    if i>10:
        break


192418
192418
38484
(96,)
0 1
1 1
2 0
3 0
4 0
5 0
6 1
7 0
8 0
9 0
10 1
11 0


In [5]:
testSet = np.array(vectors[:ratio])
testLabel = labels[:ratio]

trainSet = np.array(vectors[ratio:])
trainLabel = labels[ratio:]

print(len(testSet))
print(len(trainSet))
print(type(trainSet))

38484
153934
<class 'numpy.ndarray'>


In [6]:
for t in trainSet:
    print(t.shape)
    break

(96,)


In [7]:
vectorSize = trainSet[0].shape[0]
print(vectorSize)

96


In [8]:
import os
import re
import gc
import time
import numpy as np
import tensorflow as tf
import pandas as pd

from numpy import array
from numpy import asarray
from numpy import zeros

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend


from keras import regularizers

#from keras.models import Sequential, Model

from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import Conv2D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Reshape
from keras.layers import MaxPooling1D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling1D
from keras.layers import GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Lambda
from keras.layers.merge import concatenate


from keras.layers.advanced_activations import LeakyReLU

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from keras.optimizers import SGD, Adam, RMSprop






print("Imports work")

C:\Users\Frost\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Frost\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Frost\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Frost\AppData\Roaming\Python\Python37\site-

Imports work


Using TensorFlow backend.


<h1>Entity Recognition Training</h1>

In [9]:

Dense1Results = []
Dense2Results = []
Dense3Results = []


all_epochs_to_run  = 1
opt = 'adam'
learningRate = 0.1
epochs_to_run = all_epochs_to_run

lossType = 'binary_crossentropy'#"sparse_categorical_crossentropy" #"categorical_crossentropy" #'binary_crossentropy'

In [10]:
epochs_to_run = 100

In [11]:
input_layer = Input(shape=(vectorSize,))

#x = Dense(8000, activation='sigmoid')(input_layer)

#x = Dense(4000, activation='sigmoid')(x)

#x = Dense(2000, activation='sigmoid')(x)

#x = Dense(1000, activation='sigmoid')(x)

#x = Dense(500, activation='sigmoid')(input_layer)

x = Dense(100, activation='sigmoid')(input_layer)

x = Dense(50, activation='sigmoid')(x)

#x = GlobalMaxPooling1D()(x)

output_layer = Dense(1, activation='sigmoid')(x)

mlp1 = keras.models.Model(inputs = input_layer, outputs = output_layer)

mlp1.summary()


#compile
mlp1.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])


print()
print("Starting training...")
print()


start = time.time()

# fit the model
mlp1.fit(trainSet, trainLabel, epochs=epochs_to_run, verbose=1)

durationSecond = time.time()-start

print()
print("Training done in",durationSecond/60,"minutes or",durationSecond,"seconds now evaluating...")
print()



# evaluate the model
trainLoss, trainAccuracy = mlp1.evaluate(trainSet, trainLabel, verbose=1)
#print('Training Accuracy: %f' % (accuracy*100))

print()


# evaluate the model
testLoss, testAccuracy = mlp1.evaluate(testSet, testLabel, verbose=1)
#print('Test Accuracy: %f' % (accuracy*100))


word = "dystrophin"

prediction = mlp1.predict(np.array([word2vec(word)]))


print(prediction)


Dense1Results += [trainAccuracy,testAccuracy]

#del model

#backend.clear_session()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               9700      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 14,801
Trainable params: 14,801
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Starting training...


Epoch 1/100
153934/153934 [==============================] - 11s 70us/step - loss: 0.1942 - accuracy: 0.9263
Epoch 2/100
153934/153934 

153934/153934 [==============================] - 10s 68us/step - loss: 0.0815 - accuracy: 0.9706
Epoch 63/100
153934/153934 [==============================] - 11s 69us/step - loss: 0.0809 - accuracy: 0.9708
Epoch 64/100
153934/153934 [==============================] - 11s 71us/step - loss: 0.0813 - accuracy: 0.9712
Epoch 65/100
153934/153934 [==============================] - 11s 70us/step - loss: 0.0795 - accuracy: 0.9713
Epoch 66/100
153934/153934 [==============================] - 11s 71us/step - loss: 0.0795 - accuracy: 0.9717
Epoch 67/100
153934/153934 [==============================] - 11s 70us/step - loss: 0.0791 - accuracy: 0.9718
Epoch 68/100
153934/153934 [==============================] - 11s 69us/step - loss: 0.0789 - accuracy: 0.9716
Epoch 69/100
153934/153934 [==============================] - 11s 70us/step - loss: 0.0782 - accuracy: 0.9721
Epoch 70/100
153934/153934 [==============================] - 11s 69us/step - loss: 0.0772 - accuracy: 0.9725
Epoch 71/100
153934/153

In [12]:
epochs_to_run = 200

In [13]:
input_layer = Input(shape=(vectorSize,))

#x = Dense(8000, activation='sigmoid')(input_layer)

#x = Dense(4000, activation='sigmoid')(x)

#x = Dense(2000, activation='sigmoid')(x)

#x = Dense(1000, activation='sigmoid')(x)

#x = Dense(500, activation='sigmoid')(input_layer)

x = Dense(100, activation='sigmoid')(input_layer)

x = Dense(50, activation='sigmoid')(x)

#x = GlobalMaxPooling1D()(x)

output_layer = Dense(1, activation='sigmoid')(x)

mlp2 = keras.models.Model(inputs = input_layer, outputs = output_layer)

mlp2.summary()


#compile
mlp2.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])


print()
print("Starting training...")
print()


start = time.time()

# fit the model
mlp2.fit(trainSet, trainLabel, epochs=epochs_to_run, verbose=1)

durationSecond = time.time()-start

print()
print("Training done in",durationSecond/60,"minutes or",durationSecond,"seconds now evaluating...")
print()



# evaluate the model
trainLoss, trainAccuracy = mlp2.evaluate(trainSet, trainLabel, verbose=1)
#print('Training Accuracy: %f' % (accuracy*100))

print()


# evaluate the model
testLoss, testAccuracy = mlp2.evaluate(testSet, testLabel, verbose=1)
#print('Test Accuracy: %f' % (accuracy*100))


word = "dystrophin"

prediction = mlp2.predict(np.array([word2vec(word)]))


print(prediction)


Dense2Results += [trainAccuracy,testAccuracy]

#del model

#backend.clear_session()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               9700      
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 14,801
Trainable params: 14,801
Non-trainable params: 0
_________________________________________________________________

Starting training...

Epoch 1/200
153934/153934 [==============================] - 10s 66us/step - loss: 0.1932 - accuracy: 0.9265
Epoch 2/200
153934/153934 [==============================] - 10s 65us/step - loss: 0.1579 - accuracy: 0.9413
Epoch 3/200


153934/153934 [==============================] - 10s 67us/step - loss: 0.0771 - accuracy: 0.9720
Epoch 68/200
153934/153934 [==============================] - 10s 67us/step - loss: 0.0767 - accuracy: 0.9722
Epoch 69/200
153934/153934 [==============================] - 10s 67us/step - loss: 0.0762 - accuracy: 0.9726
Epoch 70/200
153934/153934 [==============================] - 10s 68us/step - loss: 0.0752 - accuracy: 0.9732
Epoch 71/200
153934/153934 [==============================] - 11s 68us/step - loss: 0.0749 - accuracy: 0.9732
Epoch 72/200
153934/153934 [==============================] - 10s 68us/step - loss: 0.0744 - accuracy: 0.9732
Epoch 73/200
153934/153934 [==============================] - 10s 68us/step - loss: 0.0743 - accuracy: 0.9733
Epoch 74/200
153934/153934 [==============================] - 10s 67us/step - loss: 0.0733 - accuracy: 0.9738
Epoch 75/200
153934/153934 [==============================] - 10s 68us/step - loss: 0.0736 - accuracy: 0.9737
Epoch 76/200
153934/153

153934/153934 [==============================] - 10s 68us/step - loss: 0.0508 - accuracy: 0.9817
Epoch 142/200
153934/153934 [==============================] - 10s 68us/step - loss: 0.0504 - accuracy: 0.9817
Epoch 143/200
153934/153934 [==============================] - 10s 68us/step - loss: 0.0500 - accuracy: 0.9822
Epoch 144/200
153934/153934 [==============================] - 10s 67us/step - loss: 0.0505 - accuracy: 0.9822
Epoch 145/200
153934/153934 [==============================] - 10s 68us/step - loss: 0.0502 - accuracy: 0.9821
Epoch 146/200
153934/153934 [==============================] - 10s 67us/step - loss: 0.0497 - accuracy: 0.9822
Epoch 147/200
153934/153934 [==============================] - 11s 68us/step - loss: 0.0494 - accuracy: 0.9824
Epoch 148/200
153934/153934 [==============================] - 11s 68us/step - loss: 0.0488 - accuracy: 0.9827
Epoch 149/200
153934/153934 [==============================] - 11s 69us/step - loss: 0.0492 - accuracy: 0.9823
Epoch 150/200
1

In [14]:
epochs_to_run = 300

In [15]:
input_layer = Input(shape=(vectorSize,))

#x = Dense(8000, activation='sigmoid')(input_layer)

#x = Dense(4000, activation='sigmoid')(x)

#x = Dense(2000, activation='sigmoid')(x)

#x = Dense(1000, activation='sigmoid')(x)

#x = Dense(500, activation='sigmoid')(input_layer)

x = Dense(100, activation='sigmoid')(input_layer)

x = Dense(50, activation='sigmoid')(x)

#x = GlobalMaxPooling1D()(x)

output_layer = Dense(1, activation='sigmoid')(x)

mlp3 = keras.models.Model(inputs = input_layer, outputs = output_layer)

mlp3.summary()


#compile
mlp3.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])


print()
print("Starting training...")
print()


start = time.time()

# fit the model
mlp3.fit(trainSet, trainLabel, epochs=epochs_to_run, verbose=1)

durationSecond = time.time()-start

print()
print("Training done in",durationSecond/60,"minutes or",durationSecond,"seconds now evaluating...")
print()



# evaluate the model
trainLoss, trainAccuracy = mlp3.evaluate(trainSet, trainLabel, verbose=1)
#print('Training Accuracy: %f' % (accuracy*100))

print()


# evaluate the model
testLoss, testAccuracy = mlp3.evaluate(testSet, testLabel, verbose=1)
#print('Test Accuracy: %f' % (accuracy*100))


word = "dystrophin"

prediction = mlp3.predict(np.array([word2vec(word)]))


print(prediction)


Dense3Results += [trainAccuracy,testAccuracy]

#del model

#backend.clear_session()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 96)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               9700      
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 51        
Total params: 14,801
Trainable params: 14,801
Non-trainable params: 0
_________________________________________________________________

Starting training...

Epoch 1/300
153934/153934 [==============================] - 11s 69us/step - loss: 0.1931 - accuracy: 0.9268
Epoch 2/300
153934/153934 [==============================] - 11s 70us/step - loss: 0.1576 - accuracy: 0.9414
Epoch 3/300


153934/153934 [==============================] - 11s 74us/step - loss: 0.0773 - accuracy: 0.9728
Epoch 68/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0776 - accuracy: 0.9727
Epoch 69/300
153934/153934 [==============================] - 10s 68us/step - loss: 0.0772 - accuracy: 0.9729
Epoch 70/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0766 - accuracy: 0.9730
Epoch 71/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0758 - accuracy: 0.9732
Epoch 72/300
153934/153934 [==============================] - 11s 70us/step - loss: 0.0753 - accuracy: 0.9733
Epoch 73/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0751 - accuracy: 0.9737
Epoch 74/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0744 - accuracy: 0.9740
Epoch 75/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0737 - accuracy: 0.9742
Epoch 76/300
153934/153

153934/153934 [==============================] - 10s 66us/step - loss: 0.0540 - accuracy: 0.9814
Epoch 142/300
153934/153934 [==============================] - 10s 66us/step - loss: 0.0530 - accuracy: 0.9816
Epoch 143/300
153934/153934 [==============================] - 10s 68us/step - loss: 0.0527 - accuracy: 0.9818
Epoch 144/300
153934/153934 [==============================] - 11s 69us/step - loss: 0.0523 - accuracy: 0.9823
Epoch 145/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0529 - accuracy: 0.9813
Epoch 146/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0521 - accuracy: 0.9818
Epoch 147/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0521 - accuracy: 0.9820
Epoch 148/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0523 - accuracy: 0.9821
Epoch 149/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0516 - accuracy: 0.9822
Epoch 150/300
1

153934/153934 [==============================] - 10s 68us/step - loss: 0.0410 - accuracy: 0.9860
Epoch 215/300
153934/153934 [==============================] - 10s 68us/step - loss: 0.0417 - accuracy: 0.9857
Epoch 216/300
153934/153934 [==============================] - 10s 68us/step - loss: 0.0419 - accuracy: 0.9854
Epoch 217/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0398 - accuracy: 0.9865
Epoch 218/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0418 - accuracy: 0.9855
Epoch 219/300
153934/153934 [==============================] - 10s 67us/step - loss: 0.0401 - accuracy: 0.9862
Epoch 220/300
153934/153934 [==============================] - 11s 68us/step - loss: 0.0412 - accuracy: 0.9858
Epoch 221/300
153934/153934 [==============================] - 11s 69us/step - loss: 0.0401 - accuracy: 0.9862
Epoch 222/300
153934/153934 [==============================] - 10s 68us/step - loss: 0.0406 - accuracy: 0.9859
Epoch 223/300
1

153934/153934 [==============================] - 10s 68us/step - loss: 0.0352 - accuracy: 0.9879
Epoch 288/300
153934/153934 [==============================] - 11s 69us/step - loss: 0.0350 - accuracy: 0.9881
Epoch 289/300
153934/153934 [==============================] - 11s 69us/step - loss: 0.0340 - accuracy: 0.9882
Epoch 290/300
153934/153934 [==============================] - 11s 69us/step - loss: 0.0345 - accuracy: 0.9882
Epoch 291/300
153934/153934 [==============================] - 11s 69us/step - loss: 0.0341 - accuracy: 0.9882
Epoch 292/300
153934/153934 [==============================] - 10s 68us/step - loss: 0.0341 - accuracy: 0.9882
Epoch 293/300
153934/153934 [==============================] - 10s 68us/step - loss: 0.0341 - accuracy: 0.9880
Epoch 294/300
153934/153934 [==============================] - 10s 68us/step - loss: 0.0333 - accuracy: 0.9886
Epoch 295/300
153934/153934 [==============================] - 10s 68us/step - loss: 0.0333 - accuracy: 0.9884
Epoch 296/300
1

In [16]:
from prettytable import PrettyTable

results = PrettyTable()



results.field_names = ["Accuracy",
                       "train",  "test"]#,
                       #"Train 300 epochs", "Test 300 epochs",
                       #"Train 400 epochs", "Test 400 epochs"]

'''
denseResults = [x*100 for x in denseResults]

CNNResults = [x*100 for x in CNNResults]

LSTMResults = [x*100 for x in LSTMResults]
'''




results.add_row(["Dense"] + Dense1Results)

results.add_row(["Dense"] + Dense2Results)

results.add_row(["Dense"] + Dense3Results)

print(results)

+----------+--------------------+--------------------+
| Accuracy |       train        |        test        |
+----------+--------------------+--------------------+
|  Dense   | 0.9763275384902954 | 0.9468350410461426 |
|  Dense   | 0.9849480986595154 | 0.9416381120681763 |
|  Dense   | 0.9894630312919617 | 0.940208911895752  |
+----------+--------------------+--------------------+


<h1>Practical results when applied to our list of proteins</h1>

In [17]:
results = []

for p in proteinVectors:
    results +=[  mlp2.predict(np.array([p]))[0][0] ]

print(len(results),"proteins obtained")
print()

positive = [r for r in results if r>0.5]
print(len(positive),"positive results")

96355 proteins obtained

93039 positive results


In [18]:
import joblib
joblib.dump(mlp2, 'proteinRecognitionNetwork.pkl', compress=9)
print("exported")


exported
